In [1]:
import sys
sys.path.append("/home/ubuntu/fastai/")

In [27]:
import fastai
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
from glob import glob
import pandas as pd
import helper
from tqdm import tqdm
import hashlib
from urllib import request

In [3]:
%matplotlib inline

In [4]:
PATH = "data/food-101/images/"

In [14]:
def get_data(sz, bs):
    tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_paths(path='data', num_workers=8, tfms=tfms, bs=bs)
    #return data.resize(sz, 'tmp')
    return data if sz>300 else data.resize(340, 'tmp')

In [21]:
sz = 224 # size of image
arch = resnet50 #model architecture.. model has 101 layers
bs=64 # batch size; number of images to process at one time

In [22]:
data = get_data(sz, bs)

In [23]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.fit(lrs=1e-2, n_cycle=3)

  0%|          | 0/153 [00:00<?, ?it/s]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [17]:
learn.precompute = False
learn.fit(lrs=1e-2, n_cycle=5, cycle_len=1)

epoch      trn_loss   val_loss   accuracy                  
    0      0.497538   0.308186   0.9095    
    1      0.476179   0.286418   0.9165                    
    2      0.47042    0.285373   0.9115                    
    3      0.456474   0.273783   0.922                     
    4      0.437742   0.263922   0.92                      



[array([0.26392]), 0.9199999990463257]

In [18]:
learn.unfreeze()

In [19]:
lr=1e-2
lr=np.array([lr/18,lr/6,lr/2])

In [20]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2)


  1%|▏         | 1/77 [00:05<06:22,  5.03s/it, loss=0.77]


RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58

In [28]:
def predict_class(image_path):
    trn_tfms, val_tfms = tfms_from_model(arch, sz)
    im = val_tfms(np.array(open_image(image_path)))

    preds = learn.predict_array(im[None])
    class_idx = np.argmax(preds)
    print(data.classes[class_idx],np.max(preds))
    
    img = PIL.Image.open(image_path);
    #return img
    
    return helper.get_top_n_predictions(np.exp(preds), data.classes)
    #return np.argmax(preds)

def predict_class_web(url,ext="jpg"):
    url_hash = hashlib.md5(url.encode('utf-8')).hexdigest()
    file_name = f'data/downloaded/{url_hash}.{ext}'
    f = open(file_name, 'wb')
    f.write(request.urlopen(url).read())
    f.close()
    
    return predict_class(file_name)

In [29]:
!mkdir -p data/downloaded

In [30]:
predict_class_web("https://vetstreet.brightspotcdn.com/dims4/default/3407f3b/2147483647/thumbnail/645x380/quality/90/?url=https%3A%2F%2Fvetstreet-brightspot.s3.amazonaws.com%2Ffb%2F31%2F032a6aae436a9821acda211044fb%2Fbulldog-ap-rn4myi-645.jpg")

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1518244421288/work/torch/lib/THC/generic/THCStorage.cu:58